# Taiwanese Bankrupcy

In [70]:
# imports
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from naivebayes import NaiveBayes

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline

In [71]:
# Load learn data

df = pd.read_csv("data/taiwanese-bankrupcy.csv")

In [72]:
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,1.256969e-04,0.0,0.458143,0.000725,0.0,0.147950,0.147950,0.147950,0.169141,0.311664,0.017560,0.095921,0.138736,0.022102,0.848195,0.688979,0.688979,0.217535,4.980000e+09,0.000327,0.263100,0.363725,0.002259,0.001208,0.629951,0.021266,0.207576,0.792424,0.005024,0.390284,0.006479,0.095885,0.137757,0.398036,0.086957,0.001814,0.003487,1.820926e-04,1.165007e-04,0.032903,0.034164,0.392913,0.037135,0.672775,0.166673,0.190643,0.004094,0.001997,1.473360e-04,0.147308,0.334015,0.276920,0.001036,0.676269,0.721275,0.339077,0.025592,0.903225,0.002022,0.064856,7.010000e+08,6.550000e+09,0.593831,4.580000e+08,0.671568,0.424206,0.676269,0.339077,0.126549,0.637555,0.458609,0.520382,0.312905,0.118250,0,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,2.897851e-04,0.0,0.461867,0.000647,0.0,0.182251,0.182251,0.182251,0.208944,0.318137,0.021144,0.093722,0.169918,0.022080,0.848088,0.689693,0.689702,0.217620,6.110000e+09,0.000443,0.264516,0.376709,0.006016,0.004039,0.635172,0.012502,0.171176,0.828824,0.005059,0.376760,0.005835,0.093743,0.168962,0.397725,0.064468,0.001286,0.004917,9.360000e+09,7.190000e+08,0.025484,0.0068

In [73]:
# df.isna().sum()

In [74]:
df.describe()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6.819000e+03,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6819.000000,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.0,6819.000000
mean,0.032263,0.505180,0.558625,0.553589,0.607948,0.607929,0.998755,0.797190,0.809084,0.303623,0.781381,1.995347e+09,1.950427e+09,0.467431,1.644801e+07,0.11

In [75]:
df.shape

(6819, 96)

In [76]:
# remove leading and trailing whitespace in column names
df = df.rename(columns=lambda x: x.strip())

In [77]:
df.drop('Net Income Flag', axis=1, inplace=True)

## Custom Implementation

In [40]:
metrics_our_model = {'time':{'Holdout': {'fit': 0, 'predict': 0, 'total-time': 0}, 'CV': 0},
                     'metrics': {'Holdout': {'accuracy': 0, 'precision':0, 'recall':0, 'f1':0},
                                'CV': {'accuracy': 0, 'precision':0, 'recall':0, 'f1':0}}}

In [41]:
train, test = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)
X_test = test.drop(['Bankrupt?'], axis=1)
y_test = list(test['Bankrupt?'])

In [42]:
# Holdout on our model

# fitting the model
st1 = time.time()
nb = NaiveBayes()
nb.fit(train, 'Bankrupt?')
et1 = time.time()
elapsed_time1 = et1 - st1
metrics_our_model['time']['Holdout']['fit'] = elapsed_time1

# prediction
st2 = time.time()
predictions = nb.predict(X_test)
et2 = time.time()
elapsed_time2 = et2 - st2
total_time = et2-st1

metrics_our_model['time']['Holdout']['predict'] = elapsed_time2
metrics_our_model['time']['Holdout']['total-time'] = total_time
metrics_our_model['metrics']['Holdout']['accuracy'] = accuracy_score(y_test, predictions)
metrics_our_model['metrics']['Holdout']['precision'] = precision_score(y_test, predictions, average='macro', zero_division=0)
metrics_our_model['metrics']['Holdout']['recall'] = recall_score(y_test, predictions, average='macro', zero_division=0)
metrics_our_model['metrics']['Holdout']['f1'] = f1_score(y_test, predictions, average='macro', zero_division=0)

In [43]:
# Perform the CV on our own model

# create the folds
kf5 = KFold(n_splits=5, shuffle=False)

accuracy = 0
precision = 0
recall = 0
f1 = 0

st = time.time()

for train_index, test_index in kf5.split(df):
    
    # create the train/test split properly
    train = df.iloc[train_index]
    test = df.iloc[test_index]
    X_test = test.drop(['Bankrupt?'], axis=1)
    y_test = list(test['Bankrupt?'])

    # fitting the model
    nb = NaiveBayes()
    nb.fit(train, 'Bankrupt?')
    
    # prediction
    predictions = nb.predict(X_test)
    
    accuracy += accuracy_score(y_test, predictions)
    precision += precision_score(y_test, predictions, average='macro', zero_division=0)
    recall += recall_score(y_test, predictions, average='macro', zero_division=0)
    f1 += f1_score(y_test, predictions, average='macro', zero_division=0)
    
et = time.time()    
total_time = et - st

In [45]:
accuracy = accuracy / 5
precision = precision / 5
recall = recall / 5
f1 = f1 / 5

metrics_our_model['time']['CV'] = total_time
metrics_our_model['metrics']['CV']['accuracy'] = accuracy
metrics_our_model['metrics']['CV']['precision'] = precision
metrics_our_model['metrics']['CV']['recall'] = recall
metrics_our_model['metrics']['CV']['f1'] = f1

In [46]:
metrics_our_model

{'time': {'Holdout': {'fit': 0.7178432941436768,
   'predict': 8.616829872131348,
   'total-time': 9.334673166275024},
  'CV': 41.70227837562561},
 'metrics': {'Holdout': {'accuracy': 0.10850439882697947,
   'precision': 0.49355779427359486,
   'recall': 0.48733583489681054,
   'f1': 0.10707011929007856},
  'CV': {'accuracy': 0.37936908191564667,
   'precision': 0.5055124721902536,
   'recall': 0.6389797613153773,
   'f1': 0.2742084255044523}}}

## Scikit-learn NB

In [64]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder  

metrics_sklearn_NB = {'time':{'Holdout': {'fit': 0, 'predict': 0, 'total-time': 0}, 'CV': 0},
                     'metrics': {'Holdout': {'accuracy': 0, 'precision':0, 'recall':0, 'f1':0},
                                'CV': {'accuracy': 0, 'precision':0, 'recall':0, 'f1':0}}}

In [65]:
# holdout

le = LabelEncoder()

X = df.drop(columns=['Bankrupt?'])
y = df['Bankrupt?']

y = le.fit_transform (y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

st1 = time.time()
gnb = GaussianNB()
gnb.fit(X_train, y_train)
et1 = time.time()

st2 = time.time()
predictions = gnb.predict(X_test)
et2 = time.time()

accuracy_score(y_test, predictions)

metrics_sklearn_NB['time']['Holdout']['fit'] = et1 - st1
metrics_sklearn_NB['time']['Holdout']['predict'] = et2 - st2
metrics_sklearn_NB['time']['Holdout']['total-time'] = et2 - st1
metrics_sklearn_NB['metrics']['Holdout']['accuracy'] = accuracy_score(y_test, predictions)
metrics_sklearn_NB['metrics']['Holdout']['precision'] = precision_score(y_test, predictions, average='macro', zero_division=0)
metrics_sklearn_NB['metrics']['Holdout']['recall'] = recall_score(y_test, predictions, average='macro', zero_division=0)
metrics_sklearn_NB['metrics']['Holdout']['f1'] = f1_score(y_test, predictions, average='macro', zero_division=0)

In [66]:
# Cross validation on the sklearn NB

gnb = GaussianNB()

X = df.drop(columns=['Bankrupt?'])
y = df['Bankrupt?']
y = le.fit_transform (y)

st = time.time()

accuracy_scores = cross_val_score(gnb, X, y, cv=5, scoring="accuracy")

et = time.time()

precision_scores = cross_val_score(gnb, X, y, cv=5, scoring="precision")
recall_scores = cross_val_score(gnb, X, y, cv=5, scoring="recall")
f1_scores = cross_val_score(gnb, X, y, cv=5, scoring="f1")

In [67]:
metrics_sklearn_NB['time']['CV'] = et - st
metrics_sklearn_NB['metrics']['CV']['accuracy'] = accuracy_scores.mean()
metrics_sklearn_NB['metrics']['CV']['precision'] = precision_scores.mean()
metrics_sklearn_NB['metrics']['CV']['recall'] = recall_scores.mean()
metrics_sklearn_NB['metrics']['CV']['f1'] = f1_scores.mean()

In [68]:
metrics_sklearn_NB

{'time': {'Holdout': {'fit': 0.01799297332763672,
   'predict': 0.00701594352722168,
   'total-time': 0.0250089168548584},
  'CV': 0.13686609268188477},
 'metrics': {'Holdout': {'accuracy': 0.07673509286412512,
   'precision': 0.5014758376578576,
   'recall': 0.5016025641025641,
   'f1': 0.07672428558835775},
  'CV': {'accuracy': 0.24106421706473774,
   'precision': 0.04162330411630967,
   'recall': 0.7727272727272727,
   'f1': 0.06398301805444506}}}

## Scikit-learn Mlp

In [59]:
from sklearn.neural_network import MLPClassifier

metrics_sklearn_mlp = {'time':{'Holdout': {'fit': 0, 'predict': 0, 'total-time': 0}, 'CV': 0},
                     'metrics': {'Holdout': {'accuracy': 0, 'precision':0, 'recall':0, 'f1':0},
                                'CV': {'accuracy': 0, 'precision':0, 'recall':0, 'f1':0}}}

In [60]:
# holdout

le = LabelEncoder()

X = df.drop(columns=['Bankrupt?'])
y = df['Bankrupt?']

y = le.fit_transform (y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

st1 = time.time()
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
et1 = time.time()

st2 = time.time()
predictions = mlp.predict(X_test)
et2 = time.time()

accuracy_score(y_test, predictions)

metrics_sklearn_mlp['time']['Holdout']['fit'] = et1 - st1
metrics_sklearn_mlp['time']['Holdout']['predict'] = et2 - st2
metrics_sklearn_mlp['time']['Holdout']['total-time'] = et2 - st1
metrics_sklearn_mlp['metrics']['Holdout']['accuracy'] = accuracy_score(y_test, predictions)
metrics_sklearn_mlp['metrics']['Holdout']['precision'] = precision_score(y_test, predictions, average='macro', zero_division=0)
metrics_sklearn_mlp['metrics']['Holdout']['recall'] = recall_score(y_test, predictions, average='macro', zero_division=0)
metrics_sklearn_mlp['metrics']['Holdout']['f1'] = f1_score(y_test, predictions, average='macro', zero_division=0)

In [61]:
# Cross validation on the sklearn NB

mlp = MLPClassifier()

X = df.drop(columns=['Bankrupt?'])
y = df['Bankrupt?']
y = le.fit_transform (y)

st = time.time()

accuracy_scores = cross_val_score(mlp, X, y, cv=5, scoring="accuracy")

et = time.time()

precision_scores = cross_val_score(mlp, X, y, cv=5, scoring="precision")
recall_scores = cross_val_score(mlp, X, y, cv=5, scoring="recall")
f1_scores = cross_val_score(mlp, X, y, cv=5, scoring="f1")

In [62]:
metrics_sklearn_mlp['time']['CV'] = et - st
metrics_sklearn_mlp['metrics']['CV']['accuracy'] = accuracy_scores.mean()
metrics_sklearn_mlp['metrics']['CV']['precision'] = precision_scores.mean()
metrics_sklearn_mlp['metrics']['CV']['recall'] = recall_scores.mean()
metrics_sklearn_mlp['metrics']['CV']['f1'] = f1_scores.mean()

In [63]:
metrics_sklearn_mlp

{'time': {'Holdout': {'fit': 0.5914583206176758,
   'predict': 0.012345075607299805,
   'total-time': 0.6038033962249756},
  'CV': 10.215044260025024},
 'metrics': {'Holdout': {'accuracy': 0.8812316715542522,
   'precision': 0.500538610922431,
   'recall': 0.5011726078799249,
   'f1': 0.4955554766410138},
  'CV': {'accuracy': 0.9340126467620374,
   'precision': 0.08155730787723613,
   'recall': 0.09090909090909091,
   'f1': 0.10382066056659589}}}

## Documentation/discussion

Answer these points:
-	How efficient are your algorithms
-	Performance of your algorithms regarding performance metrics for classification/regression
-	Other findings